# Imports e definições

In [297]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from scipy import signal
from scipy.io import wavfile
import os

In [95]:
def meuLPC(s,fa):
	Janela = int(round(0.03*fa))
	Avanco = int(round(0.01*fa))
	Ordem = int(round(0.001*fa))
	marcas = np.arange(0, s.shape[0]-Janela-1, Avanco)
	CP = np.zeros((Ordem, len(marcas)))
	Pot = np.zeros(len(marcas))	
	TCZ = np.zeros(len(marcas))
	S = np.zeros((Janela-Ordem-1, Ordem+1))
	cont = 0
	for k in marcas:
		saux = s[k:k+Janela]
		Pot[cont] = np.sum(saux**2)/Janela
		pz = np.logical_and(saux[1:] > 0, saux[:-1] < 0)
		TCZ[cont] = np.sum(pz)*fa/Janela
		for m in range(Ordem+1):
			S[:,m] = saux[m:-(Ordem+1-m)]
		C = np.linalg.pinv(S[:,:-1]).dot(S[:,-1])
		CP[:,cont] = C
		cont = cont +1
	return CP, Pot, TCZ

In [129]:
def specLPC(CP):
    w = np.arange(5000/fa*np.pi, 150/fa*np.pi, -50/fa*np.pi)
    P = np.zeros(len(w), CP.shape[1])
    for k in range(CP.shape[1]):
        h = [1 -np.flipud(CP[:,k])]
        for i in range(len(w)):
            P[i,k] = abs(1/np.sum(np.exp(-1j*w[i]*np.arange(len(h)))*h))
        # P[:,k] = P[:,k]/np.sum(P[:,k])
    # P = P/np.max(P)
    for k in range(len(w)):
        P[k,:] = P[k,:]/np.max(P[k,:])
    return P

In [198]:
def espectrograma(x, fa):
	p = np.arange(np.log2(80), np.log2(fa/2), 4/24)
	f = (2*np.ones(len(p)))**p
	Janela = int(np.round(0.025*fa))
	blackman = np.blackman(Janela)
	B = np.zeros((len(f), Janela), dtype=complex)
	t = np.arange(Janela)/fa
	for k in range(len(f)):
		B[k,:] = np.exp(1j*2*np.pi*f[k]*t)*blackman
	avanco = int(np.round(Janela/4))
	marcas = np.arange(0, len(x)-Janela, avanco)
	M = len(marcas)
	S = np.zeros((len(f), M))
	cont = 0
	for n in marcas:
		S[:,cont] = abs(B.dot(x[n:n+Janela]))
		cont += 1
	for canal in range(len(f)):
		S[canal,:] = S[canal,:] - np.min(S[canal,:]) + 1e-6
		S[canal,:] = (1/np.max(S))*S[canal,:]
	return S

In [250]:
n = S.shape[0]
q = np.zeros(n, dtype=str)
frase = np.empty(S.shape[1], dtype='<U'+str(n))
for col in range(S.shape[1]):		
    pp = np.nonzero(S[:,col] > 0.5)[0]
    q[pp] = "a"
    pp = np.nonzero(np.logical_and(S[:,col] > 0.25, S[:,col] <= 0.5))[0]
    q[pp] = "b"
    pp = np.nonzero(np.logical_and(S[:,col] > 0.125, S[:,col] <= 0.25))[0]
    q[pp] = "c"
    pp = np.nonzero(S[:,col] <= 0.125)[0]
    q[pp] = "d"
    palavra = ""
    for k in range(len(q)):
        palavra = palavra+q[k]
    frase[col] = palavra

In [252]:
def geraFrases(S):
	n = S.shape[0]
	q = np.zeros(n, dtype=str)
	frase = np.empty(S.shape[1], dtype='<U'+str(n))
	for col in range(S.shape[1]):		
		pp = np.nonzero(S[:,col] > 0.5)[0]
		q[pp] = "a"
		pp = np.nonzero(np.logical_and(S[:,col] > 0.25, S[:,col] <= 0.5))[0]
		q[pp] = "b"
		pp = np.nonzero(np.logical_and(S[:,col] > 0.125, S[:,col] <= 0.25))[0]
		q[pp] = "c"
		pp = np.nonzero(S[:,col] <= 0.125)[0]
		q[pp] = "d"
		palavra = ""
		for k in range(len(q)):
			palavra = palavra+q[k]
		frase[col] = palavra
	return frase

In [293]:
def lempelziv76(s):
	K = len(np.unique(s))
	N = len(s)
	L = 1
	dic = [s[0]]
	p = 1
	L = L+1
	while p+L < N:
		pos = ''.join(s[:p+L-1]).find(''.join(s[p:p+L]))
		if pos == -1:
			dic.append(''.join(s[p:p+L]))
			p = p+L
			L = 1
		else:
			L = L+1
	dic.append(''.join(s[p:]))
	# Complexidade de Lempel-Ziv, em bits por símbolo:
	#cLZ = length(dic)/C
	cLZ = len(dic)*(np.log2(len(dic))+1)/N
	return dic, cLZ

# Carregando dados

In [15]:
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Fluente\\1 - Fluente.wav'
fs, s1 = wavfile.read(path)
print(fs, s1.shape[0]/fs)
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Silabou\\1-Silabou.wav'
fs, s2 = wavfile.read(path)
print(fs, s2.shape[0]/fs)
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Soletrou\\1 - Soletrou.wav'
fs, s3 = wavfile.read(path)
print(fs, s3.shape[0]/fs)

fa = 16000
s1 = signal.resample(s1, int(s1.shape[0]*fa/fs))
s2 = signal.resample(s2, int(s2.shape[0]*fa/fs))
s3 = signal.resample(s3, int(s3.shape[0]*fa/fs))

48000 55.14
48000 60.0
48000 59.7


In [294]:
P = espectrograma(s1, fa)
frase = geraFrases(P)
dic, c = lempelziv76(frase)
print(c)

4.75425255670369


In [295]:
P = espectrograma(s2, fa)
frase = geraFrases(P)
dic, c = lempelziv76(frase)
print(c)

3.161218904228805


In [296]:
P = espectrograma(s3, fa)
frase = geraFrases(P)
dic, c = lempelziv76(frase)
print(c)

2.4517375430872232


In [301]:
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Fluente'
files = os.listdir(path)
vals = []
for file in files:
    fs, s = wavfile.read(path+'\\'+file)
    fa = 16000
    s = signal.resample(s, int(s.shape[0]*fa/fs))
    P = espectrograma(s, fa)
    frase = geraFrases(P)
    dic, c = lempelziv76(frase)
    vals.append(c)
print(vals)

[4.75425255670369, 4.906803891982043, 3.006616321680851, 4.185558759933058, 3.370511138175557, 3.6664325621267695, 3.9543777017535544]


In [302]:
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Silabou'
files = os.listdir(path)
vals = []
for file in files:
    fs, s = wavfile.read(path+'\\'+file)
    fa = 16000
    s = signal.resample(s, int(s.shape[0]*fa/fs))
    P = espectrograma(s, fa)
    frase = geraFrases(P)
    dic, c = lempelziv76(frase)
    vals.append(c)
print(vals)

[3.161218904228805, 1.7231486464807513, 2.5470621435083296, 2.027551999772481, 3.1069552046587345, 1.5808831387832014, 1.769033966887016]


In [303]:
path = 'C:\\Meu Drive\\Doutorado Unicamp\\Projeto\\github\\projeto-leitura\\Audios\\Soletrou'
files = os.listdir(path)
vals = []
for file in files:
    fs, s = wavfile.read(path+'\\'+file)
    fa = 16000
    s = signal.resample(s, int(s.shape[0]*fa/fs))
    P = espectrograma(s, fa)
    frase = geraFrases(P)
    dic, c = lempelziv76(frase)
    vals.append(c)
print(vals)

[2.4517375430872232, 2.075541477965096, 2.2077200635478484, 2.5890963375992273, 2.342405615366654, 2.230418150758207, 2.6522767622695422]
